In [1]:
%matplotlib inline

In [2]:
import os
import time
import xarray as xr
import numpy as np
from PIL import Image

import redis

In [3]:
# Python interface to redis server
r = redis.StrictRedis(host='127.0.0.1', port=6379, db=0,
                      password=os.getenv('REDIS_PASSWORD'))


In [4]:
from IPython.display import display, clear_output

In [5]:
from io import BytesIO
import base64

shape = (400, 400)
dtype = np.int32



def arr2im(arr):
    img = np.tile(255*arr[:,:,None], (1,1,3)).astype(np.uint8)
    return Image.fromarray(img)


def im2b64(image):
    buf = BytesIO()
    image.save(buf, format="PNG")
    return "data:image/png;base64,"+base64.b64encode(buf.getvalue()).decode("utf-8")


def images_list():
    while True:
        yield r.brpop("A", 10)[1]

        
def images_pubsub():
    p = r.pubsub()
    p.subscribe('A')

    for data in p.listen():
        vals = data['data']
        if isinstance(vals, int):
            continue
        yield data['data']

        
def imagestrs(gen):
    for vals in gen:
        arr = np.fromstring(vals, dtype='<i4').reshape(shape, order='F')
        yield im2b64(arr2im(arr))
        

In [6]:
images = imagestrs(images_pubsub())
def run_vis():


    from IPython.display import HTML, display


    # We'll use HTML to create a control panel with an
    # empty image and a number of navigation buttons.

    disp_html = """
    <div class="animation" align="center">
    <img id="anim_frame" width="800" height="800" src="" style="image-rendering: pixelated;"><br>
    <button onclick="play()">Play</button>
    <button onclick="nextFrame()">Next Frame</button>
    </div>


    """

    # now the javascript to drive it.  The nextFrame() and prevFrame()
    # functions will call the kernel and pull-down the frame which
    # is generated.  The play() and reverse() functions use timeouts
    # to repeatedly call nextFrame() and prevFrame().

    javascript = """
    <script type="text/Javascript">
    var count = -1;  // keep track of frame number
    var animating = 0;  // keep track of animation direction
    var timer = null;
    var kernel = IPython.notebook.kernel;

    function output(out){
        data = out.content.data["text/plain"];
        document.getElementById("anim_frame").src = data.substring(1, data.length - 1);

        timer = setTimeout(nextFrame, 0);
    }

    var callbacks = {'output' : output};

    var callbacks = {
            'iopub' : {
                 'output' : output,
        }
    }


    function play(){
        nextFrame();
    }


    function nextFrame(){
        var msg_id = kernel.execute("next(images)", callbacks, {silent:false});
    }

    setTimeout(nextFrame, 0);

    </script>
    """

    return HTML(disp_html+javascript)

In [7]:
run_vis()